In [2]:
import random
import NeuralNet as nn
import statistics as stats
import pandas as pd
#generates random population amount of random neural networks
#intialize with varying weights for each layer but the same amount
#of layers and nodes per layer
#initialized with random activation functions as well
#outputs neural nets as a list
def genRandom(population, nodes_per_layer):
    neuralNets=[]
    layers=len(nodes_per_layer)
    for p in range(0,population):
        activation_funcs=[]
        for q in range(0,layers-1):
            #we only want tanh, arctan, or constant activation
            alpha=random.randint(0,2)
            #if 2, change to 3 which is constant
            if(alpha==2):
                activation_funcs.append(3)
            else:
                activation_funcs.append(alpha)
        net= nn.NeuralNet(activation_funcs,nodes_per_layer)
        neuralNets.append(net)
    return neuralNets


In [2]:


#Initial training of random population before genetic algs do their
#thing, pass in list of neural networks and the training dictionaries
#optionally pass in iterations for back propogation to be ran
#outputs updated neural nets 
def runBackProp(neuralNets, tics_to_inputs, tics_to_outputs, its=1):
    #get all the tics
    tics=tics_to_inputs.keys()
    #for each neural network
    for a in range(0,len(neuralNets)):
        #choose amount of iterations to initially train neural nets
        for b in range(0,its):
            #for each tic
            for b in tics:
                #back propogate over each tic
                inputs=tics_to_inputs[b]
                outputs=tics_to_inputs[b]
                neuralNets[a].backProp(inputs,outputs)
    return neuralNets

In [3]:
#returns the mean absolute deviation from estimate and right
def meanAbsoluteDev(right, estimate):
    total=0
    div=len(right)
    if(div==0):
        return None
    for a in range(0,div):
        #get the absolute value and add it to the running error total
        total+=abs(right[a]-estimate[a])
    return total/div
#returns the median absolute deviation from estimate and right
def medianAbsoluteDev(right, estimate):
    even=False
    listOfDiff=[]
    length= len(right)
    if(length==0):
        return None
    #grab all the errors
    for a in range(length):
        listOfDiff.append(abs(right[a]-estimate[a]))
    #sort them
    listOfDiff=sorted(listOfDiff)
    #if even length
    if length % 2 == 0:
        val1= listOfDiff[int(length/2)]
        val2= listOfDiff[(int(length/2))-1]
        summ=val1+val2
        return summ/2
    #if odd length
    else:
        index= length/2
        index= int(index-0.5)
        return listOfDiff[index]
#get the maximum error (ie Chebyshev criteria-minimizing the maximum error)
def cheby(right, estimate):
    listOfDiff=[]
    for a in range(len(right)):
        listOfDiff.append(abs(right[a]-estimate[a]))
    return max(listOfDiff)

#get the least squares value of right and estimate
def leastSquares(right, estimate):
    listOfDiff=[]
    total=0
    for a in range(len(right)):
        #find the difference
        partial= right[a]-estimate[a]
        #square it
        square= partial*partial
        #add it to the total
        total+=square
    return total


#takes in the actual values and what the neural net outputs
#returns the fitness score for the neural net
def fitness(actualVals,output):
    mad= meanAbsoluteDev(actualVals, output)
    #med= medianAbsoluteDev(actualVals, output)
    #ls= 1/leastSquares(actualVals, output)
    #sumOfAll=mad+med+ls
    #fitScore=mad+cheb+ls
    if(mad==None):
        return None
    mad=1/mad
    return mad





















In [28]:
#Pass in single neural network and training data dictionaries
#outputs a single fitness score for the neural net
def singleScore(neuralNet, tics_to_outputs, tics_to_inputs):
    scores=[]
    #for every ticker, we grab 4 random values to try and predict
    for tic in tics_to_outputs.keys():
        calc=[]
        real=[]
        #for 4 times
        for p in range(0,4):
            #if we have ample data at this tic
            if(len(tics_to_inputs[tic])>5):
                #get a random index
                a=random.randint(0,len(tics_to_inputs[tic])-1)
                #get the real and calculate the neural net output
                calc.append(neuralNet.calculateOutput(tics_to_inputs[tic][a], single_input=True))
                real.append(tics_to_outputs[tic][a])
        if(fitness(real,calc)!=None):
            #append the fitness
            scores.append(fitness(real, calc))
    #return the mean of all the fitnesses across every ticker
    score=stats.mean(scores)
    return score

#Pass in list of neural networks and the training data dictionaries
#returns a dictionary of the neural networks and their scores 
def cumulativeScore(neuralNets, tics_to_outputs, tics_to_inputs):
    scoresDictionary={}
    for i in neuralNets:
        score=singleScore(i,tics_to_outputs, tics_to_inputs)
        scoresDictionary.update({i:score})
    return scoresDictionary

In [5]:
#returns the highest fitness score from the score dictionary
def findMostFit(scoresDictionary):
    scores=scoresDictionary.values()
    return max(scores)

#Takes a scores dictionary and normalizes them to values from
#0 to 1 and returns said dictionary
def normalizeScores(scoresDictionary):
    maxVal=findMostFit(scoresDictionary)
    for a in scoresDictionary.keys():
        score=scoresDictionary[a]
        norm=score/maxVal
        updater={a: norm}
        scoresDictionary.update(updater)
    return scoresDictionary

In [6]:


#roulette wheel selection
#input the scores dictionary of neural nets with their scores
#and the amount of parents you want
#returns a dictionary of the to be parent neural nets and their fitness scores
def selection(scoresDictionary, amount):
    selection={}
    #normalize the scores
    scoresDictionary2=normalizeScores(scoresDictionary)
    #find sum of all the scores
    valTotal=sum(scoresDictionary2.values())
    #incrementer
    val=0
    #get keys 
    keys=scoresDictionary2.keys()
    #create list of tuples
    otherList=[]
    for a in keys:
        #get probability of reproduction
        prob=scoresDictionary2[a]/valTotal
        #multiply to 100 for simplicity
        prob=prob*100
        #add to val incrementer to get range of numbers
        prob=prob+val
        #create tuple
        tup=(a,prob)
        #update the list
        otherList.append(tup)
        #update the val incrementer
        val=prob
    #loop until we have all those selected to breed
    while(len(selection)<amount):
        #get a random val between 0 and 1 and multiply by 100
        randVal=random.random()*100
        for a in range(0,len(otherList)):
            #get the value
            val=otherList[a][1]
            if(val>randVal):
                break
        #get the original score and neural network
        keyToSee=otherList[a][0]
        valToSee=scoresDictionary[keyToSee]
        #add the selection
        selection.update({keyToSee: valToSee})
    return selection


In [7]:
#Turns the inputted one dimensional matrix to a three dimensional 
#matrix with nodes_per_layer dimensions
def turnOneToThree(listInOne, nodes_per_layer):
    threeDList=[]
    counter=0
    for a in range(0,len(nodes_per_layer)-1):
        twoDList=np.random.rand(nodes_per_layer[a],nodes_per_layer[a+1])
        for b in range(0, nodes_per_layer[a]):
            oneDList=np.zeros(nodes_per_layer[a+1])
            for c in range(0,nodes_per_layer[a+1]):
                oneDList[c]=listInOne[counter]
                counter+=1
            twoDList[b]=oneDList
        threeDList.append(twoDList)
    return threeDList


#Turns the inputted three dimensional matrix into a single list
def turnThreeToOne(matrix):
    listToReturn=[]
    for a in range(0,len(matrix)):
        for b in range(0,len(matrix[a])):
            for c in range(0, len(matrix[a][b])):
                listToReturn.append(matrix[a][b][c])
    return listToReturn

In [8]:

#This function takes in a single neural network
#Mutates its weights (not activation functions)
#and returns the mutated neural net
#inversion mutation
def mutateThatJohn(neuralNetToMutate, nodes_per_layer):
    #don't mutate the activation functions
    activation_funcs=neuralNetToMutate.activation_functions
    #creates new neural network
    neuralNetToReturn= nn.NeuralNet(activation_funcs,nodes_per_layer)
    weightsOf=neuralNetToMutate.weights
    #turn weights to one dimension
    weights1d= turnThreeToOne(weightsOf)
    #get total amount of weights
    totalAmtOfWeights=len(weights1d)
    b=0
    c=0
    #loop until we get two different indices
    while c==b:
        b= random.randint(0,totalAmtOfWeights)
        c= random.randint(0,totalAmtOfWeights)
    #grab sublist from those indices
    if b<c:
        subList=weights1d[b:c]
    else:
        subList=weights1d[c:b]
    #reverse the list
    subList.reverse()
    #replace elements in the list with reversed list
    if b<c:
        weights1d[b:c]=subList
    else:
        weights1d[c:b]=subList
    #need it back in three dimensions
    weightsBack=turnOneToThree(weights1d, nodes_per_layer)
    #set mutated weights to new neural net
    neuralNetToReturn.weights=weightsBack
    #return mutated neural net
    return neuralNetToReturn


In [9]:

#inputs two neural networks and a list of the nodes per layer 
#and crosses over the activation functions and weights of the
#two neural networks and returns the network that is their child
def crossover(neuralNetwork1, neuralNetwork2, nodes_per_layer):
    #get the weights of each neural network
    weights1=neuralNetwork1.weights
    weights2=neuralNetwork2.weights
    #get the activation functions of each neural network
    activation1=neuralNetwork1.activation_functions
    activation2=neuralNetwork2.activation_functions
    #get number of activation functions
    actives=len(activation1)
    #get random int for one-point crossover of activation functions
    cross=random.randint(0,actives)
    activation_funcs=[]
    for a in range(0,actives):
        if a<cross:
            activation_funcs.append(activation1[a])
        else:
            activation_funcs.append(activation2[a])
    #create initial neural net to return
    net= nn.NeuralNet(activation_funcs,nodes_per_layer)
    #get total amount of weights
    totalAmtOfWeights=0
    for a in range(0,len(weights1)):
        for b in range(0,len(weights1[a])):
            for c in range(0,len(weights1[a][b])):
                totalAmtOfWeights+=1
    #grab two random ints for two-point crossover
    b=0
    c=0
    while b==c:
        b= random.randint(0,totalAmtOfWeights)
        c= random.randint(0,totalAmtOfWeights)
    #find the lower index
    if b<c:
        first=b
        second=c
    else:
        first=c
        second=b
    counter=0
    #crossover weights to child
    for a in range(0,len(weights1)):
        for b in range(0,len(weights1[a])):
            for c in range(0,len(weights1[a][b])):
                if counter<first:
                    net.weights[a][b][c]=weights1[a][b][c]
                elif counter>=first and counter<second:
                    net.weights[a][b][c]=weights2[a][b][c]
                else:
                    net.weights[a][b][c]=weights1[a][b][c]
                counter+=1
    #return child
    return net

In [10]:

#Takes in a dictionary of those selected to be parents
#Takes in a dictionary of all the neural nets and their scores
#Takes in a population threshold (optionally takes in proportion to mutate)
#Returns list of mutated to pass to next generation
def mutateThoseJohns(population, selectionDictionary, scoresDictionary, nodes_per_layer, mutate=0.09):
    #get the list of neural networks
    listOfPotential=list(scoresDictionary.keys())
    #get the list of the selected neural networks
    listOfSelected=list(selectionDictionary.keys())
    #find the length of the list of potential
    totalLength=len(listOfPotential)
    #set amount selected to 0
    amountForMutate=0
    #set amount to select to the proportion of the population needed to
    #be mutated
    amountToMutate=population*mutate
    #list of mutated
    listOfMutated=[]
    #loop until all have been selected for mutation
    while(amountForMutate<amountToMutate):
        #get a random index to select
        randomIndex=random.randint(0,totalLength-1)
        #select it
        neuralNetToMutate=listOfPotential[randomIndex]
        #if it was not already selected
        if(neuralNetToMutate not in listOfSelected):
            #mutate it
            neuralNetToMutate=mutateThatJohn(neuralNetToMutate, nodes_per_layer)
            #add it to the list
            listOfMutated.append(neuralNetToMutate)
            #increment
            amountForMutate+=1
    #return list
    return listOfMutated

In [11]:

#generates next generation with population as the threshold
#value for amount in the next generation and scoresDictionary that
#maps neural nets to their fitness scores.
#Returns list of next generation of neural nets
def evaluation(population, selection, scoresDictionary, nodes_per_layer):
    #start list of next population
    nextPopulation=[]
    #grab the parents we will be using
    parents=list(selection.keys())
    #mutate some of the population and append them for the next generation
    nextPopulation.extend(mutateThoseJohns(population, selection, scoresDictionary, nodes_per_layer))
    #append parents to the next population
    nextPopulation.extend(parents)
    #get list of neural networks
    thelist1 = [(k, v) for k, v in scoresDictionary.items()] 
    #find the best neural net
    best=findMostFitNeuralNet(thelist1)
    nextPopulation.append(best[0])
    #loop until we have a full next population
    while len(nextPopulation)<population: 
        #parent indices
        par1=0
        par2=0
        #until two different parents
        while par1==par2:
            par1=random.randint(0,len(parents)-1)
            par2=random.randint(0,len(parents)-1)
        #grab the parents
        parent1=parents[par1]
        parent2=parents[par2]
        #create the child
        child= crossover(parent1, parent2, nodes_per_layer)
        #append the child
        nextPopulation.append(child)
    return nextPopulation


In [12]:

#Executes iterations worth of generations of neural networks
def letsDoSomeGeneticAlgorithms(nodes_per_layer, population, tics_to_outputs, tics_to_inputs, amount=0.3,iterations=100):
    #create initial population of neural networks
    neuralNets=genRandom(population, nodes_per_layer)
    #get amount of parents wanted
    parents=amount*population
    #set loop condition
    i=0
    #after iterations amount of iterations
    while i<iterations:
        #get the scores of the neural networks
        scoresDictionary=cumulativeScore(neuralNets, tics_to_outputs, tics_to_inputs)
        #grab parents for next generation
        selectionDictionary=selection(scoresDictionary, parents)
        #create next population
        nextPop=evaluation(population, selectionDictionary, scoresDictionary, nodes_per_layer)
        #set that to neuralNets
        neuralNets=nextPop
        #increment i
        i+=1
    #return the neural network loop
    return neuralNets


In [3]:
#establish all of the tics in frame
frame=pd.read_csv("stock_names.csv")
frame['Ticker']

0          A
1        AAC
2       AACG
3        AAL
4       AAME
        ... 
5116    ZUMZ
5117     ZUO
5118     ZVO
5119    ZYNE
5120    ZYXI
Name: Ticker, Length: 5121, dtype: object

In [14]:
import pandas as pd
import numpy as np


#This function grabs all the appropriate inputs for predicting the given 
#date for the given tic
def getInputs(tic, date, inputData): 
    listOfDates= list(inputData.index)
    listOfNums=[x for x in range(0,len(listOfDates))]
    datesToNums= dict(zip(listOfDates, listOfNums))
    new_dict = dict([(value, key) for key, value in datesToNums.items()]) 
    try:
        beginningIndex= datesToNums[date]
    except:
        return -1
    endIndex=beginningIndex-20
    if(endIndex<0):
        return -1
    relevantDates=[]
    for i in range(endIndex, beginningIndex):
        relevantDates.append(new_dict[i])
    output=[]
    #append 20 closes, ranges, singleDay changes, and day to day changes
    for i in relevantDates:
        output.append(inputData['close'][i])
        output.append(inputData['range'][i])
        output.append(inputData['singleDay'][i])
        output.append(inputData['dayToDay'][i])
        #append the last days data for the rest of the stats
        if i == relevantDates[-1]:
            output.append(inputData['low'][i])
            output.append(inputData['high'][i])
            output.append(inputData['average'][i])
            output.append(inputData['open'][i])
            output.append(inputData['volume'][i])
            output.append(inputData['twelveDay'][i])
            output.append(inputData['twentySixDay'][i])
            output.append(inputData['volumeEMA'][i])
            output.append(inputData['fiftyTwoDayHigh'][i])
            output.append(inputData['fiftyTwoWeekHigh'][i])
            output.append(inputData['fiftyTwoDayLow'][i])
            output.append(inputData['fiftyTwoWeekLow'][i])
            output.append(inputData['fiftyTwoWeekAverage'][i])
            output.append(inputData['fiftyTwoDayStandDev'][i])
            output.append(inputData['fiftyTwoWeekStandDev'][i])
    return np.array(output)

#grabs all the inputs for every tic
#called inputs for back prop but is used for genetic algs also
def inputsForBackProp(tics):
    inputters=[]
    outputters=[]
    k=0
    for tic in tics:
        print(tics[k])
        k+=1
        tic=str(tic)
        try:
            #read in the training data as a dataframe
            output_values = pd.read_csv('training/' + tic + r'.csv')
            #creates an array of input vectors for a given stock and the training days
            input_values = [0]*len(output_values.index)
            data = pd.read_csv('normalized_data/' + tic + r'.csv')
            data = data.set_index('date')
            i = 0
            #for every wanted data, get the inputs 
            for date in output_values.date:
                input = getInputs(tic,date,data)
                #catches error if not enough previous days
                if type(input) ==type(-1):
                    output_values = output_values[output_values.date != date]
                    input_values = input_values[:-1]
                else:
                    input_values[i] = input
                    i = i + 1
            

            inputters.append(input_values)
            outputters.append(output_values['dayToDay'].to_numpy())
        except IOError:
            print("not here")
    #map tics to their respective lists of inputs and outputs
    input_dict=dict(zip(tics, inputters))
    output_dict=dict(zip(tics,outputters))
    #return list of both dicts
    return [input_dict, output_dict]
#same as before but uses testing data not training data
def inputsForTesting(tics):
    inputters=[]
    outputters=[]
    k=0
    for tic in tics:
        print(tics[k])
        k+=1
        tic=str(tic)
        try:
            #grab testing data
            output_values = pd.read_csv('testing/' + tic + r'.csv')
            #creates an array of input vectors for a given stock and the training days
            input_values = [0]*len(output_values.index)
            data = pd.read_csv('normalized_data/' + tic + r'.csv')
            data = data.set_index('date')
            i = 0
            for date in output_values.date:
                input = getInputs(tic,date,data)
                #catches error if not enough previous days
                if type(input) ==type(-1):
                    output_values = output_values[output_values.date != date]
                    input_values = input_values[:-1]
                else:
                    input_values[i] = input
                    i = i + 1


            inputters.append(input_values)
            outputters.append(output_values['dayToDay'].to_numpy())
        except IOError:
            print("not here")
    #map tics to their respective lists of inputs and outputs
    input_dict=dict(zip(tics, inputters))
    output_dict=dict(zip(tics,outputters))
    #return list of both dicts
    return [input_dict, output_dict]
#finds the most fit neural net from a list of tuples of neural nets and their 
#fitness scores
def findMostFitNeuralNet(thelist):
    maxer=0
    index=-1
    for a in range(0,len(thelist)):
        if(thelist[a][1]>maxer):
            maxer=thelist[a][1]
            index=a

    return thelist[index]
    

In [15]:
#how many did we get right in direction
def directionPred(actual,predicted):
    right=0;
    wrong=0;
    for i in range(0,len(actual)):
        for j in range(0,len(actual[i])):
            if(predicted[i][j]>0 and actual[i][j]>0):
                right+=1
            elif(predicted[i][j]<0 and actual[i][j]<0):
                right+=1
            elif(predicted[i][j]==0 and actual[i][j]==0):
                right+=1
            elif(actual[i][j]==0):
                #don't count as wrong because there was no direction
            else:
                wrong+=1
    return [right,wrong]
#see what the error was on average
def averageError(actual, predicted):
    error=0
    ticker=0
    for i in range(0,len(actual)):
        for j in range(0,len(actual[i])):
            val=actual[i][j]-predicted[i][j]
            if(val<0):
                val=val*(-1)
            error+=val
            ticker+=1
    return error/ticker
#grab all the best predictions for the testing data
def bestPredictions(best):
    actual=[]
    predicted=[]
    for tic in outputsForTesting[0].keys():
        calc=[]
        real=[]
        for p in range(0,len(outputsForTesting[0][tic])):
            calc.append(best[0].calculateOutput(outputsForTesting[0][tic][p], single_input=True))
            real.append(outputsForTesting[1][tic][p])
        actual.append(real)
        predicted.append(calc)
    return [actual, predicted]
#average out the output for the whole population and see how that does 
#as a prediction
from statistics import mean
def wholePopAveragePredictions(final):
    actual=[]
    predicted=[]
    for tic in outputsForTesting[0].keys():
        calc=[]
        real=[]
        for p in range(0,len(outputsForTesting[0][tic])):
            temp=[]
            #for every neural net
            for k in range(0,len(final)):
                temp.append(final[k].calculateOutput(outputsForTesting[0][tic][p], single_input=True))
            #get the mean of all the neural net predictions
            calc.append(mean(temp))
            real.append(outputsForTesting[1][tic][p])
        actual.append(real)
        predicted.append(calc)
    return [actual, predicted]


In [16]:
#grab every ticker
tics=[]
for i in range(0,len(frame['Ticker'])):
    tics.append(frame['Ticker'][i])
#get random tickers to test
def getRandomTics(n):
    i=0
    ret=[]
    while(i<n):
        #get random index
        k=random.randint(0,len(tics)-1)
        ret.append(tics[k])
        i+=1
    return ret

In [17]:
#specific testing instance
toppers=['FITB','FTEK','HD','BNDW','TTGT','NWS','DXJS','DSWL','NGG','SSNC',
        'BSMQ','TROV','YETI','PCN','ARD','TRPX','BIO.B','MIRM','TYHT','EBR.B']

In [19]:
#data setup, can choose to insert randos instead (for testing purposes)
#randos=getRandomTics(20)
outputs=inputsForBackProp(frame['Ticker'])
outputsForTesting=inputsForTesting(frame['Ticker'])


A
AAC
AACG
AAL
AAME
AAOI
AAON
AAP
AAPL
AAWW
AAXJ
AAXN
AB
ABBV
ABCB
ABDC
ABEO
ABEV
ABIO
ABM
ABMD
ABR-A
ABR-C
ABTX
ABUS
AC
ACAD
ACAM
ACAMU
ACAMW
ACB
ACBI
ACCO
ACER
ACGL
ACGLO
ACGLP
ACHC
ACHN
ACHV
ACIA
ACIU
ACIW
ACLS
ACM
ACMR
ACNB
ACOR
ACP
ACRE
ACRS
ACRX
ACST
ACT
ACTG
ACTT
ACTTU
ACTTW
ACWI
ACWX
ADAP
ADBE
ADC
ADES
ADI
ADIL
ADILW
ADMA
ADMP
ADMS
ADNT
ADP
ADPT
ADRA
ADRD
ADRE
ADRO
ADRU
ADSK
ADSW
ADTN
ADUS
ADVM
ADX
ADXN
ADXS
AEE
AEGN
AEH
AEHR
AEIS
AEM
AEMD
AEP
AER
AERI
AESE
AEY
AEYE
AEZS
AFB
AFG
AFGE
AFH
AFI
AFIN
AFINP
AFMD
AFT
AFYA
AGBAR
AGBAW
AGCO
AGEN
AGFS
AGFSW
AGI
AGIO
AGLE
AGM-A
AGM-D
AGMH
AGN
AGNC
AGNCM
AGNCN
AGNCO
AGND
AGO-B
AGO-F
AGRO
AGRX
AGTC
AGX
AGYS
AGZD
AHCO
AHH
AHL-C
AHL-E
AHPI
AHT-D
AHT-G
AHT-I
AI-B
AIA
AIC
AIG
AIG.W
AIH
AIHS
AIMC
AIMT
AINV
AIQ
AIR
AIRG
AIRR
AIRT
AIRTP
AIRTW
AIV
AIZ
AJG
AJX
AKAM
AKBA
AKCA
AKER
AKO.A
AKR
AKRO
AKRX
AKTS
AKTX
AL
ALAC
ALACR
ALACU
ALACW
ALB
ALBO
ALCO
ALDX
ALE
ALEC
ALG
ALGN
ALGR
ALGRR
ALGRU
ALGRW
ALGT
ALIM
ALJJ
ALKS
ALL
ALL-B
ALL-E
ALL-G
ALL-Y
ALLK
AL

FDUSZ
FE
FEI
FEIM
FELE
FEM
FEMB
FEMS
FENC
FENG
FEP
FET
FEUZ
FEX
FEYE
FFA
FFBC
FFBW
FFG
FFHL
FFIC
FFIN
FFIV
FFNW
FFWM
FG.W
FGBI
FGEN
FGM
FGP
FHB
FHK
FHN-A
FIBK
FICO
FID
FII
FINX
FIS
FISI
FISV
FITB
FITBI
FITBO
FITBP
FIV
FIVE
FIVN
FIXD
FIXX
FIZZ
FJP
FKO
FKU
FL
FLDM
FLEX
FLGT
FLIC
FLIR
FLL
FLLC
FLLCR
FLLCU
FLLCW
FLMN
FLMNW
FLN
FLNG
FLNT
FLOW
FLS
FLWS
FLXN
FLXS
FLY
FMAO
FMB
FMBH
FMBI
FMCI
FMCIU
FMCIW
FMHI
FMK
FMN
FMNB
FMS
FMY
FNB
FNCB
FND
FNHC
FNJN
FNK
FNKO
FNLC
FNV
FNWB
FNX
FNY
FOCS
FOF
FOLD
FOMX
FONR
FORD
FORK
FORM
FORR
FORTY
FOSL
FOX
FOXA
FOXF
FPA
FPA.U
FPAC
FPAY
FPAYW
FPH
FPI-B
FPRX
FPXE
FPXI
FR
FRAC
FRAF
FRAN
FRBA
FRBK
FRC-D
FRC-G
FRC-I
FREQ
FRG
FRGI
FRHC
FRME
FRPH
FRPT
FRSX
FRT
FRTA
FSB
FSBC
FSBW
FSCT
FSEA
FSFG
FSK
FSLR
FSM
FSRV
FSRVU
FSRVW
FSTR
FSV
FSZ
FT
FTA
FTAC
FTACU
FTACW
FTAG
FTAI
FTC
FTCS
FTDR
FTEK
FTEO
FTFT
FTGC
FTHI
FTI
FTLB
FTNT
FTR
FTRI
FTS
FTSL
FTSM
FTSV
FTV
FTXD
FTXG
FTXH
FTXL
FTXN
FTXO
FTXR
FUL
FULC
FULT
FUNC
FUND
FUSB
FUTU
FUV
FV
FVC
FVCB
FVE
FVRR
FWONA
FWONK
FWP
FWRD
F

OMER
OMEX
OMF
OMN
OMP
ON
ONB
ONCS
ONCT
ONCY
ONDK
ONEM
ONEQ
ONTX
ONTXW
ONVO
OOMA
OPB
OPBK
OPCH
OPES
OPESW
OPGN
OPHC
OPI
OPINI
OPK
OPNT
OPOF
OPRA
OPRT
OPRX
OPTN
OPTT
OPY
ORA
ORBC
ORC
ORCL
ORG
ORGO
ORGS
ORLY
ORMP
ORN
ORRF
ORSN
ORSNR
ORSNU
ORSNW
ORTX
OSBC
OSBCP
OSG
OSIS
OSLE
OSMT
OSN
OSPN
OSS
OSTK
OSUR
OSW
OTEL
OTEX
OTIC
OTLK
OTLKW
OTTR
OTTW
OVBC
OVID
OVLY
OXBR
OXBRW
OXFD
OXLC
OXLCM
OXLCO
OXM
OXSQ
OXSQL
OXSQZ
OYST
OZK
PAA
PAAC
PAACR
PAACU
PAACW
PAAS
PAC.W
PACB
PACK
PACQ
PACQU
PACQW
PACW
PAGP
PAHC
PAI
PANL
PANW
PARR
PATI
PATK
PAVM
PAVMW
PAVMZ
PAYS
PAYX
PB
PBB
PBBI
PBCT
PBCTP
PBF
PBFS
PBH
PBHC
PBI-B
PBIP
PBPB
PBR.A
PBTS
PBY
PBYI
PCAR
PCB
PCG
PCH
PCK
PCN
PCOM
PCRX
PCSB
PCTI
PCTY
PCYG
PCYO
PD
PDBC
PDCE
PDCO
PDD
PDEV
PDEX
PDFS
PDLB
PDLI
PDM
PDP
PDSB
PDT
PEB
PEB-D
PEB-F
PEBK
PEBO
PECK
PEGA
PEGI
PEI
PEI-C
PEIX
PEN
PENN
PEP
PER
PERI
PESI
PETQ
PETS
PETZ
PEY
PEZ
PFBC
PFBI
PFE
PFF
PFG
PFH
PFI
PFIE
PFIN
PFIS
PFLT
PFM
PFMT
PFN
PFPT
PFS
PFSW
PG
PGC
PGEN
PGJ
PGNX
PGNY
PGR
PGTI
PH
PHAS
PHAT
PHCF
PHG
PHIO


ZFGN
ZG
ZGNX
ZION
ZIONL
ZIONN
ZIONO
ZIONP
ZIONW
ZIOP
ZIV
ZIXI
ZKIN
ZLAB
ZM
ZN
ZNGA
ZNH
ZS
ZSAN
ZTR
ZUMZ
ZUO
ZVO
ZYNE
ZYXI
A
AAC
AACG
AAL
AAME
AAOI
AAON
AAP
AAPL
AAWW
AAXJ
AAXN
AB
ABBV
ABCB
ABDC
ABEO
ABEV
ABIO
ABM
ABMD
ABR-A
ABR-C
ABTX
ABUS
AC
ACAD
ACAM
ACAMU
ACAMW
ACB
ACBI
ACCO
ACER
ACGL
ACGLO
ACGLP
ACHC
ACHN
ACHV
ACIA
ACIU
ACIW
ACLS
ACM
ACMR
ACNB
ACOR
ACP
ACRE
ACRS
ACRX
ACST
ACT
ACTG
ACTT
ACTTU
ACTTW
ACWI
ACWX
ADAP
ADBE
ADC
ADES
ADI
ADIL
ADILW
ADMA
ADMP
ADMS
ADNT
ADP
ADPT
ADRA
ADRD
ADRE
ADRO
ADRU
ADSK
ADSW
ADTN
ADUS
ADVM
ADX
ADXN
ADXS
AEE
AEGN
AEH
AEHR
AEIS
AEM
AEMD
AEP
AER
AERI
AESE
AEY
AEYE
AEZS
AFB
AFG
AFGE
AFH
AFI
AFIN
AFINP
AFMD
AFT
AFYA
AGBAR
AGBAW
AGCO
AGEN
AGFS
AGFSW
AGI
AGIO
AGLE
AGM-A
AGM-D
AGMH
AGN
AGNC
AGNCM
AGNCN
AGNCO
AGND
AGO-B
AGO-F
AGRO
AGRX
AGTC
AGX
AGYS
AGZD
AHCO
AHH
AHL-C
AHL-E
AHPI
AHT-D
AHT-G
AHT-I
AI-B
AIA
AIC
AIG
AIG.W
AIH
AIHS
AIMC
AIMT
AINV
AIQ
AIR
AIRG
AIRR
AIRT
AIRTP
AIRTW
AIV
AIZ
AJG
AJX
AKAM
AKBA
AKCA
AKER
AKO.A
AKR
AKRO
AKRX
AKTS
AKTX
AL
ALAC
ALACR
ALACU

FCAL
FCAN
FCAP
FCAU
FCBC
FCBP
FCCO
FCCY
FCEF
FCEL
FCFS
FCN
FCNCA
FCT
FCVT
FDBC
FDEF
FDEU
FDIV
FDNI
FDS
FDT
FDTS
FDUS
FDUSG
FDUSL
FDUSZ
FE
FEI
FEIM
FELE
FEM
FEMB
FEMS
FENC
FENG
FEP
FET
FEUZ
FEX
FEYE
FFA
FFBC
FFBW
FFG
FFHL
FFIC
FFIN
FFIV
FFNW
FFWM
FG.W
FGBI
FGEN
FGM
FGP
FHB
FHK
FHN-A
FIBK
FICO
FID
FII
FINX
FIS
FISI
FISV
FITB
FITBI
FITBO
FITBP
FIV
FIVE
FIVN
FIXD
FIXX
FIZZ
FJP
FKO
FKU
FL
FLDM
FLEX
FLGT
FLIC
FLIR
FLL
FLLC
FLLCR
FLLCU
FLLCW
FLMN
FLMNW
FLN
FLNG
FLNT
FLOW
FLS
FLWS
FLXN
FLXS
FLY
FMAO
FMB
FMBH
FMBI
FMCI
FMCIU
FMCIW
FMHI
FMK
FMN
FMNB
FMS
FMY
FNB
FNCB
FND
FNHC
FNJN
FNK
FNKO
FNLC
FNV
FNWB
FNX
FNY
FOCS
FOF
FOLD
FOMX
FONR
FORD
FORK
FORM
FORR
FORTY
FOSL
FOX
FOXA
FOXF
FPA
FPA.U
FPAC
FPAY
FPAYW
FPH
FPI-B
FPRX
FPXE
FPXI
FR
FRAC
FRAF
FRAN
FRBA
FRBK
FRC-D
FRC-G
FRC-I
FREQ
FRG
FRGI
FRHC
FRME
FRPH
FRPT
FRSX
FRT
FRTA
FSB
FSBC
FSBW
FSCT
FSEA
FSFG
FSK
FSLR
FSM
FSRV
FSRVU
FSRVW
FSTR
FSV
FSZ
FT
FTA
FTAC
FTACU
FTACW
FTAG
FTAI
FTC
FTCS
FTDR
FTEK
FTEO
FTFT
FTGC
FTHI
FTI
FTLB
FTNT
FTR
FTRI
FTS
FTSL
F

ODP
ODT
OEC
OESX
OFED
OFG
OFG-B
OFIX
OFLX
OFS
OFSSI
OFSSL
OFSSZ
OGE
OGI
OHI
OIA
OII
OIIM
OKE
OKTA
OLD
OLED
OLLI
OLP
OMAB
OMCL
OMER
OMEX
OMF
OMN
OMP
ON
ONB
ONCS
ONCT
ONCY
ONDK
ONEM
ONEQ
ONTX
ONTXW
ONVO
OOMA
OPB
OPBK
OPCH
OPES
OPESW
OPGN
OPHC
OPI
OPINI
OPK
OPNT
OPOF
OPRA
OPRT
OPRX
OPTN
OPTT
OPY
ORA
ORBC
ORC
ORCL
ORG
ORGO
ORGS
ORLY
ORMP
ORN
ORRF
ORSN
ORSNR
ORSNU
ORSNW
ORTX
OSBC
OSBCP
OSG
OSIS
OSLE
OSMT
OSN
OSPN
OSS
OSTK
OSUR
OSW
OTEL
OTEX
OTIC
OTLK
OTLKW
OTTR
OTTW
OVBC
OVID
OVLY
OXBR
OXBRW
OXFD
OXLC
OXLCM
OXLCO
OXM
OXSQ
OXSQL
OXSQZ
OYST
OZK
PAA
PAAC
PAACR
PAACU
PAACW
PAAS
PAC.W
PACB
PACK
PACQ
PACQU
PACQW
PACW
PAGP
PAHC
PAI
PANL
PANW
PARR
PATI
PATK
PAVM
PAVMW
PAVMZ
PAYS
PAYX
PB
PBB
PBBI
PBCT
PBCTP
PBF
PBFS
PBH
PBHC
PBI-B
PBIP
PBPB
PBR.A
PBTS
PBY
PBYI
PCAR
PCB
PCG
PCH
PCK
PCN
PCOM
PCRX
PCSB
PCTI
PCTY
PCYG
PCYO
PD
PDBC
PDCE
PDCO
PDD
PDEV
PDEX
PDFS
PDLB
PDLI
PDM
PDP
PDSB
PDT
PEB
PEB-D
PEB-F
PEBK
PEBO
PECK
PEGA
PEGI
PEI
PEI-C
PEIX
PEN
PENN
PEP
PER
PERI
PESI
PETQ
PETS
PETZ
PEY
PEZ
PFBC
PFBI
PFE

YGYIP
YI
YIN
YJ
YLCO
YLDE
YMAB
YNDX
YORW
YPF
YRCW
YTEN
YTRA
YUM
YVR
YY
Z
ZAGG
ZAYO
ZB-G
ZBH
ZBIO
ZBRA
ZEAL
ZEN
ZEUS
ZFGN
ZG
ZGNX
ZION
ZIONL
ZIONN
ZIONO
ZIONP
ZIONW
ZIOP
ZIV
ZIXI
ZKIN
ZLAB
ZM
ZN
ZNGA
ZNH
ZS
ZSAN
ZTR
ZUMZ
ZUO
ZVO
ZYNE
ZYXI


In [29]:
#Script to get results on finished neural net population
#Set the nodes per layer which implicitly sets the amount of layers
nodes_per_layer=[95,2,2,2,1]
#Get the final population of neural nets
final=letsDoSomeGeneticAlgorithms(nodes_per_layer, 100, outputs[1], outputs[0])
#find their fitness scores

In [30]:
#see how this instance did
a=cumulativeScore(final, outputs[1], outputs[0])
#put this dictionary into a list of tuples
thelist1 = [(k, v) for k, v in a.items()] 
#find the best neural net
best=findMostFitNeuralNet(thelist1)
#Find predictions on the best one and the population as a whole
bestPreds=bestPredictions(best)
popPreds=wholePopAveragePredictions(final)
#Find Stats
bestStats=directionPred(bestPreds[0],bestPreds[1])
bestError=averageError(bestPreds[0],bestPreds[1])
avStats=directionPred(popPreds[0],popPreds[1])
avError=averageError(popPreds[0],popPreds[1])
#Report Stats
print("Here is how the most fit neural net faired on averaged: ")
print(bestError)
print("Here is how the most fit neural net faired directionally: ")
print("Right: ")
print(bestStats[0])
print("Wrong: ")
print(bestStats[1])
print("Here is how the neural net pop faired on averaged: ")
print(avError)
print("Here is how the neural net pop faired directionally: ")
print("Right: ")
print(avStats[0])
print("Wrong: ")
print(avStats[1])


Here is how the most fit neural net faired on averaged: 
0.019852264660590837
Here is how the most fit neural net faired directionally: 
Right: 
2131081
Wrong: 
2542421
Here is how the neural net pop faired on averaged: 
0.020111037965984933
Here is how the neural net pop faired directionally: 
Right: 
2130963
Wrong: 
2542539


TypeError: 'NeuralNet' object is not subscriptable

In [21]:
#function to export npy file
from tempfile import TemporaryFile
def exportNeuralNet(best, name):
    path= name + '.npy';
    np.save(path,best[0].weights)

In [32]:
#export to npy file
exportNeuralNet(best,"wholePop")

In [62]:
#do what we did above 30 times for parameter results
bestStats=[]
bestError=[]
avStats=[]
avError=[]
for touchdown in range(0,30):
    #Script to get results on finished neural net population
    #Set the nodes per layer which implicitly sets the amount of layers
    nodes_per_layer=[95,2,2,1]
    #Get the final population of neural nets
    final=letsDoSomeGeneticAlgorithms(nodes_per_layer, 60, outputs[1], outputs[0])
    #find their fitness scores
    a=cumulativeScore(final, outputs[1], outputs[0])
    #put this dictionary into a list of tuples
    thelist1 = [(k, v) for k, v in a.items()] 
    #find the best neural net
    best=findMostFitNeuralNet(thelist1)
    #Find predictions on the best one and the population as a whole
    bestPreds=bestPredictions(best)
    popPreds=wholePopAveragePredictions(final)
    #Find Stats
    bestStats.append(directionPred(bestPreds[0],bestPreds[1]))
    bestError.append(averageError(bestPreds[0],bestPreds[1]))
    print(touchdown)
    print(bestError[touchdown])
    #print(best[0].activation_functions)
    avStats.append(directionPred(popPreds[0],popPreds[1]))
    avError.append(averageError(popPreds[0],popPreds[1]))
#Report Stats
right=0
righto=0
for yes in range(0,len(bestStats)):
    right+=bestStats[yes][0]
for no in range(0,len(avStats)):
    righto+=avStats[no][0]
right=right/len(bestStats)
righto=righto/len(avStats)
print("Here is how the most fit neural net faired on averaged: ")
print(sum(bestError)/len(bestError))
print("Here is how the most fit neural net faired directionally: ")
print("Right: ")
print(right)
print("Here is how the neural net pop faired on averaged: ")
print(sum(avError)/len(avError))
print("Here is how the neural net pop faired directionally: ")
print("Right: ")
print(righto)

0
0.018643216602908592
1
0.019731048308783226
2
0.018642862337350602
3
0.019167725906991954
4
0.018581531705030036
5
0.018644591737265372
6
0.018748607350100133
7
0.018736136145931497
8
0.018600299543731044
9
0.021809725587367487
10
0.018571152904827946
11
0.018664865334698404
12
0.019047313413352335
13
0.018755070060157086
14
0.021216582649173372
15
0.018687583470255273
16
0.01857314588511644
17
0.018587915727446842
18
0.01865898321307124
19
0.01862903896876358
20
0.01863938472027482
21
0.019701945787023778
22
0.018673166220605997
23
0.01894006991833922
24
0.01882087088556777
25
0.01868850699747071
26
0.01918898244755953
27
0.018779609027092296
28
0.01856863502083402
29
0.01878364994395644
Here is how the most fit neural net faired on averaged: 
0.018982740594034906
Here is how the most fit neural net faired directionally: 
Right: 
8083.9
Here is how the neural net pop faired on averaged: 
0.019046764777421534
Here is how the neural net pop faired directionally: 
Right: 
7944.16666666

In [45]:
#see the activation functions
best[0].activation_functions

[3, 3, 3]